In [10]:
from pathlib import Path
import pickle
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Nadam
from keras.callbacks import ModelCheckpoint, TensorBoard, LearningRateScheduler
from models.models import *
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [11]:
# Number of searches performed
hyperparameter_iterations = 10
# epochs*(1frozen + 2unfrozen)
epochs = 15
# Batch size too small, bad approximation of the global loss.
# Too large, gets stuck in minima.
batch_size = 32

# Default input size for many architectures
image_size = (224, 224)

In [12]:
# Dataset folder, expects three subdirectories:
# validation, train, test
dataset_name = 'UTKFace_Split'
#dataset_name = 'Adience_Split'
dataset = Path(dataset_name)

val_test_gen = ImageDataGenerator(rescale=1./255)
validation = val_test_gen.flow_from_directory(dataset / 'validation',
                                              target_size = image_size,
                                              batch_size = batch_size,
                                              class_mode = 'categorical', shuffle=False)

test = val_test_gen.flow_from_directory(dataset / 'test',
                                        target_size = image_size,
                                        batch_size = 1,
                                        class_mode = 'categorical', shuffle=False)
validation_steps = int(np.ceil(validation.n/batch_size))

Found 2372 images belonging to 2 classes.
Found 1186 images belonging to 2 classes.


In [ ]:
# Random search
for _ in range(hyperparameter_iterations):
    # Hyperparameters
    filter_factor = np.random.randint(256,1024)
    l2_factor = np.random.uniform(0,1e-3)
    rotation_range = np.random.randint(0, 360)
    shear_range = np.random.uniform(0, 360)
    brightness = np.random.uniform(10)
    brightness_range = (-brightness, brightness)
    
    # Hyperparameter MobileNet
    alpha = np.random.choice([0.25, 0.50, 0.75,1.0])

    # Architecture
    architecture = MobileNetGenderFConnected(image_size, alpha, filter_factor, l2_factor)
    model_save_name = f'{dataset_name}_{architecture.name}_rotation_{rotation_range}_shear_{shear_range:.2E}_brightness_{brightness:.2E}'
    log_output = os.path.join(dataset_name, os.path.join('logs', model_save_name))
    model_save_name += '{val_loss:.3f}.hdf5'
    os.makedirs(log_output, exist_ok=True)
    tb = TensorBoard(log_output)
    mc = ModelCheckpoint(os.path.join('Saved models', model_save_name), save_best_only=True, save_weights_only=True)

    callbacks = [tb, mc]
    train_gen = ImageDataGenerator(rescale=1./255,
                             horizontal_flip=True,
                             shear_range=10,
                             rotation_range=10)


    train = train_gen.flow_from_directory(dataset /  'train',
                                          target_size = image_size,
                                          batch_size = batch_size,
                                          class_mode = 'categorical')
    total = train.n
    males = np.count_nonzero(train.classes) 
    females = total - males
    weights = {0: males/females, 1: 1} # If males outnumber females, we get higher loss for females
    model = architecture.model
    model.compile(optimizer=Nadam(),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    steps_per_epoch = int(np.ceil(total/batch_size))
    wrapper = lambda x : model.fit_generator(train, 
                                        epochs=x, 
                                        steps_per_epoch=steps_per_epoch, 
                                        validation_data=validation, 
                                        validation_steps=validation_steps,
                                        class_weight=weights,
                                        callbacks=callbacks)

    history_frozen = wrapper(epochs)

    # Unfreezing all layers
    for layer in enumerate(model.layers):
        layer.trainable = True

    # Learning rate is low so that we don't ruin our parameters
    model.compile(optimizer=Nadam(lr=2e-6), # default is 2e-3
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history_unfrozen = wrapper(epochs*2)
    
    test_loss, test_acc = model.evaluate_generator(test, steps=test.n, verbose=1)
    with open(os.path.join(log_output, 'histories'), 'wb') as f:
        pickle.dump([history_frozen.history, history_unfrozen.history, test_loss, test_acc], f) 
    print(f'{model_save_name}\nreached an accuracy of {test_acc} and loss of {test_loss}')